In [1]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\loicl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [5]:
train_data_dir = r'..\..\spectro_fonctionnels\Train'
test_data_dir = r'..\..\spectro_fonctionnels\Test'
img_width, img_height = 336, 336

In [6]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32 
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 7642 images belonging to 2 classes.
Found 1910 images belonging to 2 classes.


In [4]:
# On définit l'architecture du modèle :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# On compile le modeles :

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# On affiche le model :

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 334, 334, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 334, 334, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 167, 167, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 165, 165, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 165, 165, 32)      0         
__________________________________________

In [5]:
# On définit les parametres pour l'entrainement :
epochs = 14
train_samples = 7642
test_samples = 1910

In [6]:
historique = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=test_samples// batch_size)

Epoch 1/14
238/238 [==============================] - 1038s 4s/step - loss: 0.5493 - acc: 0.7435 - val_loss: 0.4217 - val_acc: 0.8300
Epoch 2/14
238/238 [==============================] - 443s 2s/step - loss: 0.4033 - acc: 0.8326 - val_loss: 0.2943 - val_acc: 0.8835
Epoch 3/14
238/238 [==============================] - 510s 2s/step - loss: 0.3116 - acc: 0.8774 - val_loss: 0.3055 - val_acc: 0.8824
Epoch 4/14
238/238 [==============================] - 437s 2s/step - loss: 0.2632 - acc: 0.8974 - val_loss: 0.2251 - val_acc: 0.9174
Epoch 5/14
238/238 [==============================] - 413s 2s/step - loss: 0.2270 - acc: 0.9126 - val_loss: 0.2087 - val_acc: 0.9206
Epoch 6/14
238/238 [==============================] - 429s 2s/step - loss: 0.2098 - acc: 0.9198 - val_loss: 0.1962 - val_acc: 0.9311
Epoch 7/14
238/238 [==============================] - 668s 3s/step - loss: 0.1939 - acc: 0.9296 - val_loss: 0.1868 - val_acc: 0.9354
Epoch 8/14
238/238 [==============================] - 803s 3s/step -

In [7]:
model.save(r'save/model_336')